<div align="center"><img src="./images/DLI_Header.png"></div>

# The NVSHMEM Memory Model

In this notebook you are going to learn about *symmetric memory* in NVSHMEM, and utilize it for your final refactor of the monte-carlo π approximation code.

## Objectives

By the time you complete this notebook you will:

- Be able to allocate and work with NVSHMEM symmetric memory.

## Symmetric Memory

Our next goal will be to average the results across all PEs, which should result in a more accurate answer. We could do this manually using standard Linux inter-process communication techniques, but NVSHMEM already offers a mechanism to do that implicitly. In order to take advantage of it, we need to learn about NVSHMEM's memory allocation API, `nvshmem_malloc()`.

This allocator works somewhat like standard `cudaMalloc`, but `cudaMalloc` returns a private address that is local to that GPU[<sup>6</sup>](#footnote6). An object allocated with `nvshmem_malloc()` is called a **symmetric data object**. Every symmetric data object has a corresponding data object with the same name, type, and size on all PEs. The virtual address corresponding to the pointer returned by `nvshmem_malloc()` is called a **symmetric address**. Symmetric addresses are legal to use in NVSHMEM communication routines for remote access on other PEs (and can also be used directly for local memory accesses by the PE that allocated each instance of the symmetric object). We can manipulate virtual addresses just like normal local addresses. When we want to use an NVSHMEM API to access the copy of a symmetric data object on a remote PE, we index into the pointer normally, and the corresponding location in the target remote PE is used. For example, if we do...

```cpp
int* a = (int*) nvshmem_malloc(sizeof(int));
```

...we can meaningfully ask for the value of `a[0]` both on the local PE with local memory accesses, and on remote PEs with remote memory accesses. One way to think about this could be that, given `M` PEs, we are distributing an array of length `M` to all PEs such that each only has one element. Since the symmetric data object in this case has length 1, we only access `a[0]` on any PE.

<center><img src="images/NVSHMEM_PGAS.png" width="1000"></center>

Dynamic memory allocations in NVSHMEM of symmetric data objects come from a special memory region called the **symmetric heap**, which is created by NVSHMEM for you during program execution[<sup>7</sup>](#footnote7) and then drawn from for all subsequent dynamic memory allocations.

## Exercise: Use Symmetric Memory in MC π Code

So we need to make three changes, which are reflected in [exercises/nvshmem_pi_step3.cpp](exercises/nvshmem_pi_step3.cpp).

*First*, we replace our call to `cudaMalloc` with `nvshmem_malloc()`. We can still use `atomicAdd()` locally on the allocated data so that the copy of the symmetric object on each PE has the same result as before.

```cpp
int* d_hits = (int*) nvshmem_malloc(sizeof(int));
```

*Second*, we sum up the results across all PEs. This is a [collective operation](https://en.wikipedia.org/wiki/Collective_operation), in particular the [reduce](https://en.wikipedia.org/wiki/Collective_operation#Reduce) operation. In NVSHMEM, we can sum all instances of a symmetric object with [nvshmem_int_sum_reduce(team, dest, source, nreduce)](https://docs.nvidia.com/nvshmem/api/gen/api/collectives.html#sum). The `source` is the symmetric address we want to sum up; the `destination` is where the results should be stored; `nreduce` is the number of elements to reduce over (just one for us, since our data is a scalar), and the `team` is the group of PEs to sum over[<sup>8</sup>](#footnote8) (we'll use the default team `NVSHMEM_TEAM_WORLD` which is the set of all PEs). Putting that together, we want to do:

```cpp
// Accumulate the results across all PEs
int* d_hits_total = (int*) nvshmem_malloc(sizeof(int));
nvshmem_int_sum_reduce(NVSHMEM_TEAM_WORLD, d_hits_total, d_hits, 1);
```

<center><img src="images/NVSHMEM_reduction.png" width="500"></center>

Now all PEs have the sum of the hits counter across all PEs, so the *third* change we need to make is to only print out the result on a single PE. By convention, we typically do this sort of thing on PE 0.

```cpp
if (my_pe == 0) {
    // Copy final result back to the host
    ...

    // Calculate final value of pi
    ...

    // Print out result
    ...
}
```

Make the relevant changes in [exercises/nvshmem_pi_step3.cpp](exercises/nvshmem_pi_step3.cpp). As usual consult the [solution](solutions/nvshmem_pi_step3.cpp) if you need help.

In [ ]:
!nvcc -x cu -arch=sm_70 -rdc=true -I $NVSHMEM_HOME/include -L $NVSHMEM_HOME/lib -lnvshmem -lcuda -o nvshmem_pi_step3 exercises/nvshmem_pi_step3.cpp
!nvshmrun -np $NUM_DEVICES ./nvshmem_pi_step3

## Next

You have spent a fair amount of time working and reworking the monte-carlo π simulation to gain exposure to several multi-GPU programming paradigms, including NVSHMEM. Now that you are familiar with the basics of NVSHMEM we are going to shift our focus onto less trivial applications, and explore how to leverage NVSHMEM for efficient communication across multiple GPUs.

Please open the next notebook: [_NVSHMEM Histogram: Duplicated Approach_](10_Histogram-Dup.ipynb).

## Footnotes

<span id="footnote6">6</span>: The exception is that within the confines of a system with NVLink-connected GPUs, it's possible to use the [CUDA IPC mechanism](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#interprocess-communication) for the GPUs to directly access each others' memory.

<span id="footnote7">7</span>: The default size of the symmetric heap is 1 GB, and this can be controlled through the environment variable [NVSHMEM_SYMMETRIC_SIZE](https://docs.nvidia.com/nvshmem/api/gen/env.html#c.NVSHMEM_SYMMETRIC_SIZE).

<span id="footnote8">8</span>: The use of "teams" to specify operations involving groups of PEs is [new to NVSHMEM 2.0](https://developer.nvidia.com/blog/accelerating-nvshmem-2-0-team-based-collectives-using-nccl/), based on the OpenSHMEM 1.5 specification.